In [1]:
import chess
import chess.uci
import chess.pgn
import os
import csv
from numpy import array
import numpy as np
import pandas as pd
import math
import statistics
import ast
import pandas as pd
import json
import fastai

# Data prep methods

In [18]:
def get_perf_df(event = 'Rated Classical game'):
    df = pd.read_csv("/Users/tylerahlstrom/Documents/GitHub/DI_proposal/stockfish_performances_DC.csv")
    df = df.drop(df[df.event != event].index)
    return df

In [3]:
def split_perfs(joint_perf_df):
    new_headers = ['elo', 'chosen_evals', 'option_evals', 'opp_elo', 'win', 'acc_name']
    split_df = pd.DataFrame(columns = new_headers)
    for index, row in joint_perf_df.iterrows():
        if len(row['result']) is 3:
            split_df = split_df.append({'elo': row['elo_w'], 'chosen_evals' : row['chosen_moves_eval_w'], 'option_evals' : row['available_moves_eval_w'], 'opp_elo': row['elo_b'], 'result': row['result'][0], 'acc_name': row['acc_name_w']}, ignore_index=True)
            split_df = split_df.append({'elo': row['elo_b'], 'chosen_evals' : row['chosen_moves_eval_b'], 'option_evals' : row['available_moves_eval_b'], 'opp_elo': row['elo_w'], 'result': row['result'][2], 'acc_name': row['acc_name_b']}, ignore_index=True)
    return split_df

In [4]:
def convert_json_to_list(df):
    for index, row in df.iterrows():
        row['chosen_evals'] = json.loads(row['chosen_evals'])
        row['option_evals'] = json.loads(row['option_evals'])
    return df

In [5]:
def get_list_of_chosen_moves(dict_of_move_dict): #e.g. {u'11': {u'move_rank': 2, u'cp_scor#
    lol_of_moves = []
    for key, d_move in dict_of_move_dict.items():
        lol_of_moves.append([key, d_move])
    lol_of_moves.sort(key=lambda x: int(x[0]))
    return lol_of_moves

In [22]:
def get_list_of_list_of_available_moves(dict_of_options_dict): # e.g. {u'24': {u'd7e8': {u'cp_score': -674, u'mate_s...
    lolol_of_options  = []
    for key, d_options in dict_of_options_dict.items():
        lol_of_options = []
        for key2, d_option in d_options.items():
            lol_of_options.append([key2, d_option])
        lol_of_options.sort(key=lambda x: int(x[1]['rank']))
    
        lolol_of_options.append([key, lol_of_options])
    lolol_of_options.sort(key=lambda x: int(x[0][0]))
    return lolol_of_options

In [24]:
def get_list_of_rank_percentiles(list_of_moves):
    list_of_rank_percentiles = []
    for move in list_of_moves:
        rank = int(move[1]['move_rank'])
        num_options = int(move[1]['num_move_options'])
        chunk = float(1)/float(num_options)
        percentile = 1.0 - (float(rank-1) * chunk)
        list_of_rank_percentiles.append(percentile)
    return list_of_rank_percentiles

In [6]:
def get_list_of_move_cps(list_of_moves):
    list_of_cps = []
    for move in list_of_moves:
        cp = move[1]['cp_score']
        list_of_cps.append(cp)
    return list_of_cps

In [7]:
def get_list_of_move_mates(list_of_moves):
    list_of_mates = []
    for move in list_of_moves:
        mate = move[1]['mate_score']
        list_of_mates.append(mate)
    return list_of_mates

In [8]:
def get_list_of_option_cps(list_of_av_moves):
    #print(list_of_av_moves)
    lol_of_option_cps = []
    for move in list_of_av_moves:
        options_cps = []
        for option in move[1]:
            options_cps.append(option[1]['cp_score'])
        lol_of_option_cps.append(options_cps)
    
    #print(lol_of_option_cps)
    return lol_of_option_cps

In [74]:
def get_list_of_dist_percentiles(move_cps, option_cps):
    dist_scores = []
    for i in range(len(move_cps)):
        cp_temp = [x for x in option_cps[i] if x != None]
        max_cp = None
        min_cp = None
        if (len(cp_temp) > 0):
            max_cp = max(cp_temp)
            min_cp = min(cp_temp)
    
        #avg_cp = sum([x for x in option_cps[i] if x is not None])/float((len([x for x in option_cps[i] if x is not None])+0.1))
        if move_cps[i] is None:
            move_cps[i] = -2000
        if max_cp is None:
            max_cp = -10
        if min_cp is None:
            min_cp = -200
        
        if max_cp == min_cp:
            dist_scores.append(0.5)
            continue
        dist = max(0, 1- (abs(move_cps[i])/abs(max_cp-min_cp)))#move_cps[i] - avg_cp
        dist_scores.append(dist)
        #percentile = float(better_than_cp)/min(float(total_cp), -1)
        #dist_percentiles.append(percentile)
    
    return dist_scores

In [75]:
def get_raw_data_df(event):
    data_df = get_perf_df(event)
    data_df = data_df.drop_duplicates()
    data_df = data_df.sample(frac=1).reset_index(drop=True)
    data_df = split_perfs(data_df)
    data_df = convert_json_to_list(data_df)
    data_df = data_df.sample(frac=1).reset_index(drop=True)
    return data_df


In [76]:
def get_elo_system_prediction(result, opp_elo):
    k_factor = 40
    base_elo = 1560
    Ea = 1./(1.+10.**((opp_elo - base_elo)/400.))
    Rnew = base_elo + k_factor*(float(result) - Ea)
    return Rnew

In [77]:
def remove_short_games(X, y):
    i = 0
    while i < (len(X)):
        if len(X[i]['cps']) < 30:
            X.pop(i)
            y.pop(i)
            i -= 1
        i+=1
    return X, y
    

In [78]:
def get_desired_data(complete_data_df, to_select = ['rank_percentiles', 'dist_percentiles', 'cps', 'result', 'opp_elo', 'acc_name', 'mates']):
    i=0
    X_selected_ldl = [] #X_selected_ldl is a list of dictionaries of lists, easiest way (i think) to track all the relevant data
    y = []#elo targets
    for index, row in complete_data_df.iterrows():
        if i == 0:
            print(index, row)
        i += 1
        row_dict = {}
        
        ch_moves = get_list_of_chosen_moves(row['chosen_evals'])
        av_moves = get_list_of_list_of_available_moves(row['option_evals'])
        
        if 'rank_percentiles' in to_select:
            rank_percentiles = get_list_of_rank_percentiles(ch_moves)
            row_dict['rank_percentiles'] = rank_percentiles
        if 'cps' in to_select: #TO ADD: cp percentiles (e.g., just how much worse would the worst move have been?)
            cps = get_list_of_move_cps(ch_moves)
            row_dict['cps'] = cps
        if 'dist_percentiles' in to_select:
            option_cps = get_list_of_option_cps(av_moves)
            dis_percentiles = get_list_of_dist_percentiles(cps, option_cps)
            row_dict['dist_percentiles'] = dis_percentiles
        if 'opp_elo' in to_select:
            row_dict['opp_elo'] = row['opp_elo']
        if 'result' in to_select:
            row_dict['result'] = row['result']
        if 'acc_name' in to_select:
            row_dict['acc_name'] = row['acc_name']
        if 'mates' in to_select:
            mates = get_list_of_move_mates(ch_moves)
            row_dict['mates'] = mates
        
        elo = row['elo']
        X_selected_ldl.append(row_dict)
        y_entry = []
        y_entry.append(elo)
        elo_system_prediction = get_elo_system_prediction(row_dict['result'],row_dict['opp_elo'])
        y_entry.append(int(elo_system_prediction))
        y.append(y_entry)

    return X_selected_ldl, y #X_selected_ldl is a list of dictionaries of lists

In [96]:
def TabularizeMates(X_raw):
    for x in X_raw:
        found_mate = []
        continued_mate = []
        lost_mate = []
        moved_into_mate = []
        continued_being_mated = []

        for m in x['mates']:  
            found = 0
            continued = 0
            lost = 0
            found_bad = 0
            continued_bad = 0

            if m != None:
                if (m[0:2] == 'AB'):
                    found = 1
                if (m[0:2] == 'AC'):
                    continued = 1
                if (m[0:2] == 'AL'):
                    lost = 1
                if (m[0:2] == 'DB'):
                    found_bad = 1
                if (m[0:2] == 'DC'):
                    continued_bad = 1    
            found_mate.append(found)
            continued_mate.append(continued)
            lost_mate.append(lost)
            moved_into_mate.append(found_bad)
            continued_being_mated.append(continued_bad)

        x['found_mate'] = found_mate
        x['continued_mate'] = continued_mate
        x['lost_mate'] = lost_mate
        x['moved_into_mate'] = moved_into_mate
        x['continued_being_mated'] = continued_being_mated
    


In [104]:
def RefineX(X_):
    refined_X = []
    for x in X_:
        g = []
        for i in range(len(x['cps'])):
            m = []
            m.append(min(10, x['cps'][i]))
            m.append(x['dist_percentiles'][i])
            m.append(x['rank_percentiles'][i])
            m.append(x['found_mate'][i])
            m.append(x['continued_mate'][i])
            m.append(x['lost_mate'][i])
            m.append(x['moved_into_mate'][i])
            m.append(x['continued_being_mated'][i])
            g.append(m)
        refined_X.append(g)
    return refined_X
    
    

In [158]:
def CreateLables(X_, Y_):
    lables = []
    for i in range(len(X_)):
        single_lable = []
        for _ in range(len(X_[i]['cps'])):
            single_lable.append(Y_[i][0]) #the second item in Y[i] is the standard system prediction
        lables.append(single_lable)
    return lables
        

In [79]:
def get_data(event = 'Rated Classical game'):
    data_df = get_raw_data_df(event)
    X, y = get_desired_data(data_df)
    X, y = remove_short_games(X, y)
    return X, y

# Get the data

In [153]:
X_raw, y_raw = get_data(event = 'Rated Classical game')

0 elo                                                          1305
chosen_evals    {'0': {'move': 'e2e4', 'move_rank': 1, 'num_mo...
option_evals    {'0': {'e2e4': {'rank': 1, 'cp_score': -22, 'm...
opp_elo                                                      1320
win                                                           NaN
acc_name                                             goodplayer93
result                                                          1
Name: 0, dtype: object


In [154]:
X_raw[0]

{'acc_name': 'corporaldonut',
 'cps': [-22,
  -37,
  -57,
  -60,
  -58,
  -324,
  -5,
  -42,
  -141,
  -44,
  -17,
  -198,
  -46,
  -83,
  0,
  -440,
  -132,
  -7,
  0,
  -15,
  -70,
  -364,
  -8,
  -18,
  25,
  -2000,
  -4742,
  -18,
  -14,
  -49,
  -5,
  -492,
  230,
  -917,
  -2000,
  -2000,
  -2000],
 'dist_percentiles': [0.875,
  0.9175946547884187,
  0.8926553672316384,
  0.9112426035502958,
  0.9307048984468339,
  0.7631578947368421,
  0.9963503649635036,
  0.9778364116094986,
  0.8109919571045576,
  0.9305993690851735,
  0.983284169124877,
  0.8123222748815166,
  0.9098039215686274,
  0.9236430542778289,
  1.0,
  0.7109067017082786,
  0.899390243902439,
  0.9947565543071161,
  1.0,
  0.9888309754281459,
  0.9581589958158996,
  0.7659163987138263,
  0.9938744257274119,
  0.9781818181818182,
  0.9773755656108597,
  0,
  0,
  0.9918330308529946,
  0.9943865276663993,
  0.7421052631578947,
  0.5,
  0,
  0.7032258064516129,
  0,
  0,
  0,
  0],
 'mates': [None,
  None,
  None,
  Non

In [159]:
TabularizeMates(X_raw)
X_ = RefineX(X_raw)
X = np.array([np.array(xi) for xi in X_])
Y_ = CreateLables(X_raw, y_raw)
Y = np.array([np.array(yi) for yi in Y_])

In [162]:
X[2].shape

(33, 8)

In [163]:
Y[2].shape

(33,)

In [127]:
Y[0]

[[1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581],
 [1535, 1581]]

In [141]:
X_raw[0]

{'acc_name': 'Xoal',
 'cps': [-17,
  -19,
  -81,
  -25,
  -327,
  -295,
  -85,
  -127,
  -196,
  -14,
  -26,
  -69,
  -439,
  -4,
  -36,
  -27,
  -156,
  -34,
  -223,
  -40,
  -2000,
  -2000,
  -26,
  -30,
  -9,
  -14,
  -28,
  -67,
  -203,
  -63,
  -1673,
  48,
  -548,
  -2000,
  -2,
  -40],
 'dist_percentiles': [0.891025641025641,
  0.9828519855595668,
  0.9662077596996246,
  0.9794745484400657,
  0.7394422310756972,
  0.7675334909377463,
  0.9348659003831418,
  0.9361488185017597,
  0.8460329929300864,
  0.9883817427385893,
  0.8452380952380952,
  0.9307228915662651,
  0.6189236111111112,
  0.9983606557377049,
  0.8105263157894737,
  0.9786223277909739,
  0.7891891891891891,
  0.7462686567164178,
  0.956983024691358,
  0.9722222222222222,
  0,
  0.6239894717052077,
  0.995077622112836,
  0.972875226039783,
  0.9945913461538461,
  0.9891975308641975,
  0.9933030375508252,
  0.9854821235102925,
  0.5,
  0.21250000000000002,
  0,
  0.9402985074626866,
  0.5791090629800307,
  0,
  0.991

In [190]:
Y[0].shape

(37,)

In [174]:
X[0].shape

(37, 8)

# Build the model

In [198]:
from keras import Sequential
from keras.layers import Dense, LSTM
from keras.utils import to_categorical

In [313]:
model = Sequential()
model.add(LSTM(units=50, dropout = 0.3, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=50, dropout = 0.3, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 30, 50)            11800     
_________________________________________________________________
lstm_53 (LSTM)               (None, 30, 50)            20200     
_________________________________________________________________
lstm_54 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 51        
Total params: 52,251
Trainable params: 52,251
Non-trainable params: 0
_________________________________________________________________


In [317]:
model.fit(X_trimmed_train, Y_trimmed_train, epochs=800, batch_size=32, validation_data=(X_trimmed_test, Y_trimmed_test))

Train on 3333 samples, validate on 700 samples
Epoch 1/200
3333/3333 [==============================] - 5s 2ms/step - loss: 339824.1829 - val_loss: 340384.2305
Epoch 2/200
3333/3333 [==============================] - 6s 2ms/step - loss: 334659.8549 - val_loss: 335216.4898
Epoch 3/200
3333/3333 [==============================] - 6s 2ms/step - loss: 329556.0507 - val_loss: 330109.7271
Epoch 4/200
3333/3333 [==============================] - 5s 2ms/step - loss: 324510.9020 - val_loss: 325080.9311
Epoch 5/200
3333/3333 [==============================] - 6s 2ms/step - loss: 319514.7916 - val_loss: 320064.2132
Epoch 6/200
3333/3333 [==============================] - 6s 2ms/step - loss: 314562.4425 - val_loss: 315114.6329
Epoch 7/200
3333/3333 [==============================] - 5s 2ms/step - loss: 309658.6476 - val_loss: 310220.6270
Epoch 8/200
3333/3333 [==============================] - 6s 2ms/step - loss: 304807.9936 - val_loss: 305359.8162
Epoch 9/200
3333/3333 [==========================

3333/3333 [==============================] - 5s 2ms/step - loss: 98789.4176 - val_loss: 99277.9290
Epoch 73/200
3333/3333 [==============================] - 6s 2ms/step - loss: 97114.6669 - val_loss: 97607.1917
Epoch 74/200
3333/3333 [==============================] - 6s 2ms/step - loss: 95487.0546 - val_loss: 95976.1718
Epoch 75/200
3333/3333 [==============================] - 6s 2ms/step - loss: 93900.2639 - val_loss: 94382.4938
Epoch 76/200
3333/3333 [==============================] - 6s 2ms/step - loss: 92363.5391 - val_loss: 92840.1822
Epoch 77/200
3333/3333 [==============================] - 6s 2ms/step - loss: 90867.2743 - val_loss: 91349.2008
Epoch 78/200
3333/3333 [==============================] - 6s 2ms/step - loss: 89401.0257 - val_loss: 89888.3326
Epoch 79/200
3333/3333 [==============================] - 6s 2ms/step - loss: 87989.6545 - val_loss: 88463.0533
Epoch 80/200
3333/3333 [==============================] - 6s 2ms/step - loss: 86613.5537 - val_loss: 87085.7158
Epoch

Epoch 145/200
3333/3333 [==============================] - 5s 2ms/step - loss: 59535.2985 - val_loss: 59806.1896
Epoch 146/200
3333/3333 [==============================] - 5s 2ms/step - loss: 59532.1631 - val_loss: 59802.3877
Epoch 147/200
3333/3333 [==============================] - 5s 2ms/step - loss: 59532.2126 - val_loss: 59797.8238
Epoch 148/200
3333/3333 [==============================] - 5s 2ms/step - loss: 59530.4294 - val_loss: 59796.5341
Epoch 149/200
3333/3333 [==============================] - 5s 2ms/step - loss: 59529.6497 - val_loss: 59795.3472
Epoch 150/200
3333/3333 [==============================] - 5s 1ms/step - loss: 59528.5794 - val_loss: 59795.3809
Epoch 151/200
3333/3333 [==============================] - 5s 1ms/step - loss: 59527.0939 - val_loss: 59793.3045
Epoch 152/200
3333/3333 [==============================] - 5s 1ms/step - loss: 59527.0841 - val_loss: 59791.7524
Epoch 153/200
3333/3333 [==============================] - 5s 1ms/step - loss: 59526.5133 - val_

In [257]:
Xt = X

In [261]:
Xt[0][0:2]

array([[-22.        ,   0.875     ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [-37.        ,   0.91759465,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ]])

In [269]:
X_trimmed = []
for each in Xt:
    X_trimmed.append(each[0:30])

In [271]:
X_trimmed = np.array(X_trimmed)

In [272]:
X_trimmed.shape

(4033, 30, 8)

In [281]:
Y_trimmed = []
for each in Y:
    Y_trimmed.append(each[0:1])

In [282]:
Y_trimmed = np.array(Y_trimmed)

In [283]:
Y_trimmed.shape

(4033, 1)

In [318]:
X_trimmed_train = X_trimmed[:3800]
X_trimmed_test = X_trimmed[3800:]
Y_trimmed_train = Y_trimmed[:3800]
Y_trimmed_test = Y_trimmed[3800:]

model = Sequential()
model.add(LSTM(units=50, dropout = 0.3, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=50, dropout = 0.3, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_trimmed_train, Y_trimmed_train, epochs=800, batch_size=32, validation_data=(X_trimmed_test, Y_trimmed_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_55 (LSTM)               (None, 30, 50)            11800     
_________________________________________________________________
lstm_56 (LSTM)               (None, 30, 50)            20200     
_________________________________________________________________
lstm_57 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 51        
Total params: 52,251
Trainable params: 52,251
Non-trainable params: 0
_________________________________________________________________
Train on 3633 samples, validate on 400 samples
Epoch 1/800
3633/3633 [==============================] - 9s 2ms/step - loss: 2470330.6026 - val_loss: 2461056.0000
Epoch 2/800
3633/3633 [==============================] - 5s 1ms/step - loss: 2445193.6722 - val_loss: 244237

Epoch 64/800
3633/3633 [==============================] - 5s 1ms/step - loss: 1515175.5196 - val_loss: 1514441.8700
Epoch 65/800
3633/3633 [==============================] - 5s 1ms/step - loss: 1502237.6261 - val_loss: 1501524.4300
Epoch 66/800
3633/3633 [==============================] - 5s 1ms/step - loss: 1489363.3451 - val_loss: 1488661.7100
Epoch 67/800
3633/3633 [==============================] - 5s 1ms/step - loss: 1476545.5187 - val_loss: 1475886.3900
Epoch 68/800
3633/3633 [==============================] - 5s 1ms/step - loss: 1463793.3004 - val_loss: 1463137.9150
Epoch 69/800
3633/3633 [==============================] - 5s 1ms/step - loss: 1451098.6546 - val_loss: 1450447.1800
Epoch 70/800
3633/3633 [==============================] - 5s 1ms/step - loss: 1438468.7706 - val_loss: 1437854.8600
Epoch 71/800
3633/3633 [==============================] - 5s 1ms/step - loss: 1425898.8823 - val_loss: 1425297.4350
Epoch 72/800
3633/3633 [==============================] - 5s 1ms/step - 

Epoch 135/800
3633/3633 [==============================] - 5s 1ms/step - loss: 745104.1514 - val_loss: 745731.0800
Epoch 136/800
3633/3633 [==============================] - 5s 1ms/step - loss: 736391.5810 - val_loss: 737027.7375
Epoch 137/800
3633/3633 [==============================] - 5s 1ms/step - loss: 727738.1167 - val_loss: 728385.7000
Epoch 138/800
3633/3633 [==============================] - 5s 1ms/step - loss: 719142.7392 - val_loss: 719811.4250
Epoch 139/800
3633/3633 [==============================] - 5s 1ms/step - loss: 710604.7989 - val_loss: 711307.2350
Epoch 140/800
3633/3633 [==============================] - 5s 1ms/step - loss: 702124.0035 - val_loss: 702837.8625
Epoch 141/800
3633/3633 [==============================] - 5s 1ms/step - loss: 693703.5964 - val_loss: 694428.9175
Epoch 142/800
3633/3633 [==============================] - 5s 1ms/step - loss: 685341.3723 - val_loss: 686095.0875
Epoch 143/800
3633/3633 [==============================] - 5s 1ms/step - loss: 6

3633/3633 [==============================] - 5s 1ms/step - loss: 270735.5517 - val_loss: 272669.0625
Epoch 207/800
3633/3633 [==============================] - 5s 1ms/step - loss: 266113.3983 - val_loss: 268047.0812
Epoch 208/800
3633/3633 [==============================] - 5s 1ms/step - loss: 261543.0858 - val_loss: 263498.9387
Epoch 209/800
3633/3633 [==============================] - 5s 1ms/step - loss: 257032.1893 - val_loss: 259009.8050
Epoch 210/800
3633/3633 [==============================] - 5s 1ms/step - loss: 252575.7393 - val_loss: 254565.1800
Epoch 211/800
3633/3633 [==============================] - 5s 1ms/step - loss: 248172.4843 - val_loss: 250187.1788
Epoch 212/800
3633/3633 [==============================] - 5s 1ms/step - loss: 243826.6637 - val_loss: 245852.8588
Epoch 213/800
3633/3633 [==============================] - 5s 1ms/step - loss: 239535.0986 - val_loss: 241582.6250
Epoch 214/800
3633/3633 [==============================] - 5s 1ms/step - loss: 235302.8091 - v

3633/3633 [==============================] - 5s 1ms/step - loss: 74919.1150 - val_loss: 77982.2038
Epoch 278/800
3633/3633 [==============================] - 5s 1ms/step - loss: 73924.2770 - val_loss: 77001.1138
Epoch 279/800
3633/3633 [==============================] - 5s 1ms/step - loss: 72971.6519 - val_loss: 76059.8897
Epoch 280/800
3633/3633 [==============================] - 5s 1ms/step - loss: 72059.5361 - val_loss: 75166.1034
Epoch 281/800
3633/3633 [==============================] - 5s 1ms/step - loss: 71185.5993 - val_loss: 74300.2828
Epoch 282/800
3633/3633 [==============================] - 5s 1ms/step - loss: 70351.9797 - val_loss: 73485.3584
Epoch 283/800
3633/3633 [==============================] - 5s 1ms/step - loss: 69556.4526 - val_loss: 72687.1053
Epoch 284/800
3633/3633 [==============================] - 5s 1ms/step - loss: 68799.2662 - val_loss: 71951.6094
Epoch 285/800
3633/3633 [==============================] - 5s 1ms/step - loss: 68083.2483 - val_loss: 71237.09

...

3633/3633 [==============================] - 5s 1ms/step - loss: 59240.1182 - val_loss: 62559.3644
Epoch 494/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.0520 - val_loss: 62559.6866
Epoch 495/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.6842 - val_loss: 62559.4594
Epoch 496/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59238.8348 - val_loss: 62559.9150
Epoch 497/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59240.0498 - val_loss: 62559.5875
Epoch 498/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.3833 - val_loss: 62559.5178
Epoch 499/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.4978 - val_loss: 62559.8281
Epoch 500/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.5223 - val_loss: 62559.5891
Epoch 501/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59238.9981 - val_loss: 62559.43

3633/3633 [==============================] - 5s 1ms/step - loss: 59241.5674 - val_loss: 62559.1791
Epoch 566/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59241.2659 - val_loss: 62559.7088
Epoch 567/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.5490 - val_loss: 62559.5312
Epoch 568/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59240.5918 - val_loss: 62559.8725
Epoch 569/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.1731 - val_loss: 62559.4016
Epoch 570/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59238.9099 - val_loss: 62559.1691
Epoch 571/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59238.6877 - val_loss: 62559.3625
Epoch 572/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59238.9508 - val_loss: 62559.3628
Epoch 573/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.4584 - val_loss: 62559.35

3633/3633 [==============================] - 5s 1ms/step - loss: 59238.7521 - val_loss: 62559.3925
Epoch 638/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59238.8923 - val_loss: 62559.6778
Epoch 639/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.8409 - val_loss: 62559.9097
Epoch 640/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.3315 - val_loss: 62559.8425
Epoch 641/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.6080 - val_loss: 62559.2831
Epoch 642/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59238.5275 - val_loss: 62559.7069
Epoch 643/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59240.6848 - val_loss: 62559.2862
Epoch 644/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59241.1319 - val_loss: 62559.6394
Epoch 645/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.4522 - val_loss: 62559.90

3633/3633 [==============================] - 5s 1ms/step - loss: 59238.5464 - val_loss: 62559.5881
Epoch 710/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.9203 - val_loss: 62559.2950
Epoch 711/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.2781 - val_loss: 62559.6475
Epoch 712/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.2185 - val_loss: 62559.8500
Epoch 713/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59241.2310 - val_loss: 62559.6994
Epoch 714/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.3168 - val_loss: 62559.4050
Epoch 715/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59242.8802 - val_loss: 62559.7228
Epoch 716/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59239.2973 - val_loss: 62559.2972
Epoch 717/800
3633/3633 [==============================] - 5s 1ms/step - loss: 59238.8227 - val_loss: 62559.38

KeyboardInterrupt: 

In [320]:
X_trimmed_train = X_trimmed[:3800]
X_trimmed_test = X_trimmed[3800:]
Y_trimmed_train = Y_trimmed[:3800]
Y_trimmed_test = Y_trimmed[3800:]

model = Sequential()
model.add(LSTM(units=50, dropout = 0.3, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=50, dropout = 0.3, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_trimmed_train, Y_trimmed_train, epochs=500, batch_size=32, validation_data=(X_trimmed_test, Y_trimmed_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_61 (LSTM)               (None, 30, 50)            11800     
_________________________________________________________________
lstm_62 (LSTM)               (None, 30, 50)            20200     
_________________________________________________________________
lstm_63 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 51        
Total params: 52,251
Trainable params: 52,251
Non-trainable params: 0
_________________________________________________________________
Train on 3733 samples, validate on 300 samples
Epoch 1/500
3733/3733 [==============================] - 10s 3ms/step - loss: 2471544.6542 - val_loss: 2440976.7433
Epoch 2/500
3733/3733 [==============================] - 6s 2ms/step - loss: 2445354.5615 - val_loss: 24210

Epoch 64/500
3733/3733 [==============================] - 5s 1ms/step - loss: 1461620.7793 - val_loss: 1444352.1200
Epoch 65/500
3733/3733 [==============================] - 5s 1ms/step - loss: 1448097.8204 - val_loss: 1430920.4650
Epoch 66/500
3733/3733 [==============================] - 5s 1ms/step - loss: 1434640.1339 - val_loss: 1417576.6400
Epoch 67/500
3733/3733 [==============================] - 5s 1ms/step - loss: 1421253.5136 - val_loss: 1404272.3317
Epoch 68/500
3733/3733 [==============================] - 5s 1ms/step - loss: 1407930.6129 - val_loss: 1391068.0117
Epoch 69/500
3733/3733 [==============================] - 5s 1ms/step - loss: 1394678.8709 - val_loss: 1377899.4783
Epoch 70/500
3733/3733 [==============================] - 5s 1ms/step - loss: 1381491.2150 - val_loss: 1364809.3350
Epoch 71/500
3733/3733 [==============================] - 5s 1ms/step - loss: 1368375.2817 - val_loss: 1351792.6067
Epoch 72/500
3733/3733 [==============================] - 5s 1ms/step - 

Epoch 135/500
3733/3733 [==============================] - 5s 1ms/step - loss: 669889.8328 - val_loss: 659401.1775
Epoch 136/500
3733/3733 [==============================] - 5s 1ms/step - loss: 661164.9613 - val_loss: 650773.4525
Epoch 137/500
3733/3733 [==============================] - 5s 1ms/step - loss: 652502.3136 - val_loss: 642210.2825
Epoch 138/500
3733/3733 [==============================] - 6s 1ms/step - loss: 643908.8314 - val_loss: 633696.3058
Epoch 139/500
3733/3733 [==============================] - 5s 1ms/step - loss: 635376.7771 - val_loss: 625263.8850
Epoch 140/500
3733/3733 [==============================] - 5s 1ms/step - loss: 626912.2666 - val_loss: 616890.0925
Epoch 141/500
3733/3733 [==============================] - 5s 1ms/step - loss: 618513.7614 - val_loss: 608590.1908
Epoch 142/500
3733/3733 [==============================] - 5s 1ms/step - loss: 610184.0012 - val_loss: 600345.6908
Epoch 143/500
3733/3733 [==============================] - 5s 1ms/step - loss: 6

3733/3733 [==============================] - 5s 1ms/step - loss: 213147.8750 - val_loss: 209184.3944
Epoch 207/500
3733/3733 [==============================] - 5s 1ms/step - loss: 209028.2664 - val_loss: 205146.2383
Epoch 208/500
3733/3733 [==============================] - 5s 1ms/step - loss: 204969.8659 - val_loss: 201190.4952
Epoch 209/500
3733/3733 [==============================] - 5s 1ms/step - loss: 200974.2460 - val_loss: 197285.6481
Epoch 210/500
3733/3733 [==============================] - 5s 1ms/step - loss: 197035.6424 - val_loss: 193445.0106
Epoch 211/500
3733/3733 [==============================] - 5s 1ms/step - loss: 193164.3808 - val_loss: 189640.9606
Epoch 212/500
3733/3733 [==============================] - 5s 1ms/step - loss: 189350.8971 - val_loss: 185918.3575
Epoch 213/500
3733/3733 [==============================] - 5s 1ms/step - loss: 185604.7003 - val_loss: 182263.0965
Epoch 214/500
3733/3733 [==============================] - 5s 1ms/step - loss: 181918.2706 - v

Epoch 278/500
3733/3733 [==============================] - 5s 1ms/step - loss: 62625.0207 - val_loss: 64184.2435
Epoch 279/500
3733/3733 [==============================] - 5s 1ms/step - loss: 62271.8471 - val_loss: 63878.8509
Epoch 280/500
3733/3733 [==============================] - 5s 1ms/step - loss: 61946.2604 - val_loss: 63603.5790
Epoch 281/500
3733/3733 [==============================] - 5s 1ms/step - loss: 61649.8113 - val_loss: 63352.0084
Epoch 282/500
3733/3733 [==============================] - 5s 1ms/step - loss: 61379.9492 - val_loss: 63121.6133
Epoch 283/500
3733/3733 [==============================] - 5s 1ms/step - loss: 61133.6551 - val_loss: 62919.8904
Epoch 284/500
3733/3733 [==============================] - 5s 1ms/step - loss: 60911.6031 - val_loss: 62741.2310
Epoch 285/500
3733/3733 [==============================] - 5s 1ms/step - loss: 60711.2638 - val_loss: 62577.5904
Epoch 286/500
3733/3733 [==============================] - 5s 1ms/step - loss: 60531.0009 - val_

3733/3733 [==============================] - 5s 1ms/step - loss: 59394.2613 - val_loss: 61733.8629
Epoch 351/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59396.1738 - val_loss: 61733.7652
Epoch 352/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59394.8678 - val_loss: 61733.7967
Epoch 353/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.0705 - val_loss: 61732.1050
Epoch 354/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59396.4255 - val_loss: 61732.4825
Epoch 355/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.4850 - val_loss: 61733.2000
Epoch 356/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59394.9051 - val_loss: 61733.8763
Epoch 357/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59394.2934 - val_loss: 61732.8842
Epoch 358/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.1714 - val_loss: 61732.52

3733/3733 [==============================] - 5s 1ms/step - loss: 59395.1684 - val_loss: 61735.1686
Epoch 423/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.0558 - val_loss: 61733.0984
Epoch 424/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59396.2021 - val_loss: 61733.8043
Epoch 425/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.0100 - val_loss: 61732.1062
Epoch 426/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.4249 - val_loss: 61734.9033
Epoch 427/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59396.4574 - val_loss: 61733.6152
Epoch 428/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.6383 - val_loss: 61733.4572
Epoch 429/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59394.9738 - val_loss: 61732.8597
Epoch 430/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.4409 - val_loss: 61731.58

3733/3733 [==============================] - 5s 1ms/step - loss: 59394.4441 - val_loss: 61734.7327
Epoch 495/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59394.2188 - val_loss: 61732.9484
Epoch 496/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.2642 - val_loss: 61732.9455
Epoch 497/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.1394 - val_loss: 61733.4227
Epoch 498/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59394.5460 - val_loss: 61732.8048
Epoch 499/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59395.6440 - val_loss: 61734.0371
Epoch 500/500
3733/3733 [==============================] - 5s 1ms/step - loss: 59397.5242 - val_loss: 61731.5265


In [321]:
X_trimmed_train = X_trimmed[:3800]
X_trimmed_test = X_trimmed[3800:]
Y_trimmed_train = Y_trimmed[:3800]
Y_trimmed_test = Y_trimmed[3800:]

model = Sequential()
model.add(LSTM(units=50, dropout = 0.3, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=50, dropout = 0.3, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_trimmed_train, Y_trimmed_train, epochs=500, batch_size=32, validation_data=(X_trimmed_test, Y_trimmed_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_64 (LSTM)               (None, 30, 50)            11800     
_________________________________________________________________
lstm_65 (LSTM)               (None, 30, 50)            20200     
_________________________________________________________________
lstm_66 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 51        
Total params: 52,251
Trainable params: 52,251
Non-trainable params: 0
_________________________________________________________________
Train on 3800 samples, validate on 233 samples
Epoch 1/500
3800/3800 [==============================] - 9s 2ms/step - loss: 2472527.2700 - val_loss: 2458688.0408
Epoch 2/500
3800/3800 [==============================] - 5s 1ms/step - loss: 2448322.6521 - val_loss: 243994

Epoch 64/500
3800/3800 [==============================] - 5s 1ms/step - loss: 1515980.5071 - val_loss: 1511252.0923
Epoch 65/500
3800/3800 [==============================] - 5s 1ms/step - loss: 1503008.6474 - val_loss: 1498318.1888
Epoch 66/500
3800/3800 [==============================] - 5s 1ms/step - loss: 1490095.8203 - val_loss: 1485463.7795
Epoch 67/500
3800/3800 [==============================] - 5s 1ms/step - loss: 1477246.5968 - val_loss: 1472650.5156
Epoch 68/500
3800/3800 [==============================] - 5s 1ms/step - loss: 1464453.2150 - val_loss: 1459908.4667
Epoch 69/500
3800/3800 [==============================] - 5s 1ms/step - loss: 1451722.3900 - val_loss: 1447213.1609
Epoch 70/500
3800/3800 [==============================] - 5s 1ms/step - loss: 1439050.8397 - val_loss: 1434589.5826
Epoch 71/500
3800/3800 [==============================] - 5s 1ms/step - loss: 1426440.4266 - val_loss: 1422016.2178
Epoch 72/500
3800/3800 [==============================] - 5s 1ms/step - 

Epoch 135/500
3800/3800 [==============================] - 5s 1ms/step - loss: 743782.1536 - val_loss: 742174.2782
Epoch 136/500
3800/3800 [==============================] - 5s 1ms/step - loss: 735047.8709 - val_loss: 733476.2991
Epoch 137/500
3800/3800 [==============================] - 5s 1ms/step - loss: 726372.7320 - val_loss: 724838.4423
Epoch 138/500
3800/3800 [==============================] - 5s 1ms/step - loss: 717756.7011 - val_loss: 716267.9611
Epoch 139/500
3800/3800 [==============================] - 5s 1ms/step - loss: 709197.2270 - val_loss: 707763.3812
Epoch 140/500
3800/3800 [==============================] - 5s 1ms/step - loss: 700699.3901 - val_loss: 699303.7516
Epoch 141/500
3800/3800 [==============================] - 5s 1ms/step - loss: 692260.8641 - val_loss: 690902.9565
Epoch 142/500
3800/3800 [==============================] - 6s 1ms/step - loss: 683878.6187 - val_loss: 682571.8149
Epoch 143/500
3800/3800 [==============================] - 5s 1ms/step - loss: 6

3800/3800 [==============================] - 5s 1ms/step - loss: 268735.0643 - val_loss: 270145.1825
Epoch 207/500
3800/3800 [==============================] - 5s 1ms/step - loss: 264112.4346 - val_loss: 265562.2417
Epoch 208/500
3800/3800 [==============================] - 5s 1ms/step - loss: 259547.4782 - val_loss: 261040.2902
Epoch 209/500
3800/3800 [==============================] - 5s 1ms/step - loss: 255038.2914 - val_loss: 256577.8169
Epoch 210/500
3800/3800 [==============================] - 5s 1ms/step - loss: 250587.1492 - val_loss: 252154.4481
Epoch 211/500
3800/3800 [==============================] - 5s 1ms/step - loss: 246193.0723 - val_loss: 247801.2509
Epoch 212/500
3800/3800 [==============================] - 5s 1ms/step - loss: 241853.8554 - val_loss: 243513.4295
Epoch 213/500
3800/3800 [==============================] - 5s 1ms/step - loss: 237569.7605 - val_loss: 239260.5327
Epoch 214/500
3800/3800 [==============================] - 6s 1ms/step - loss: 233340.4905 - v

3800/3800 [==============================] - 6s 2ms/step - loss: 74068.5177 - val_loss: 78182.7949
Epoch 278/500
3800/3800 [==============================] - 6s 1ms/step - loss: 73102.4405 - val_loss: 77256.6571
Epoch 279/500
3800/3800 [==============================] - 6s 1ms/step - loss: 72178.9119 - val_loss: 76353.2742
Epoch 280/500
3800/3800 [==============================] - 6s 1ms/step - loss: 71294.0154 - val_loss: 75495.5267
Epoch 281/500
3800/3800 [==============================] - 6s 1ms/step - loss: 70450.4143 - val_loss: 74691.6758
Epoch 282/500
3800/3800 [==============================] - 6s 1ms/step - loss: 69646.9315 - val_loss: 73908.4625
Epoch 283/500
3800/3800 [==============================] - 6s 2ms/step - loss: 68878.8125 - val_loss: 73175.4777
Epoch 284/500
3800/3800 [==============================] - 6s 2ms/step - loss: 68151.0038 - val_loss: 72473.8721
Epoch 285/500
3800/3800 [==============================] - 6s 2ms/step - loss: 67459.4074 - val_loss: 71812.45

3800/3800 [==============================] - 5s 1ms/step - loss: 59284.6876 - val_loss: 64197.7888
Epoch 350/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.7495 - val_loss: 64198.0375
Epoch 351/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.3399 - val_loss: 64197.8214
Epoch 352/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.7621 - val_loss: 64198.0630
Epoch 353/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.0404 - val_loss: 64198.2703
Epoch 354/500
3800/3800 [==============================] - 6s 1ms/step - loss: 59284.4799 - val_loss: 64198.2179
Epoch 355/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.5643 - val_loss: 64197.9696
Epoch 356/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.7790 - val_loss: 64197.9558
Epoch 357/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.0143 - val_loss: 64198.73

3800/3800 [==============================] - 5s 1ms/step - loss: 59285.2224 - val_loss: 64198.0387
Epoch 422/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.1473 - val_loss: 64198.2625
Epoch 423/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.2172 - val_loss: 64198.4745
Epoch 424/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.5765 - val_loss: 64198.4626
Epoch 425/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.7299 - val_loss: 64198.6375
Epoch 426/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.5213 - val_loss: 64198.6797
Epoch 427/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.5199 - val_loss: 64198.2625
Epoch 428/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59286.2057 - val_loss: 64198.6432
Epoch 429/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.3406 - val_loss: 64198.28

3800/3800 [==============================] - 5s 1ms/step - loss: 59284.6110 - val_loss: 64198.4848
Epoch 494/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.3656 - val_loss: 64198.2949
Epoch 495/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.7002 - val_loss: 64198.3983
Epoch 496/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.9447 - val_loss: 64198.4911
Epoch 497/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.0530 - val_loss: 64198.2984
Epoch 498/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.1511 - val_loss: 64198.7984
Epoch 499/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59285.0970 - val_loss: 64198.3849
Epoch 500/500
3800/3800 [==============================] - 5s 1ms/step - loss: 59284.2493 - val_loss: 64198.5409


In [336]:
X_trimmed_train = X_trimmed[:3800]
X_trimmed_test = X_trimmed[3800:]
Y_trimmed_train = Y_trimmed[:3800]
Y_trimmed_test = Y_trimmed[3800:]

model = Sequential()
model.add(LSTM(units=100, dropout = 0.5, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=100, dropout = 0.5, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=100, dropout = 0.5, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=100, dropout = 0.5))
model.add(Dense(units=1))
model.summary()
model.compile(optimizer='adam', loss='mean_absolute_error')

model.fit(X_trimmed_train, Y_trimmed_train, epochs=1000, batch_size=64, validation_data=(X_trimmed_test, Y_trimmed_test), shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_96 (LSTM)               (None, 30, 100)           43600     
_________________________________________________________________
lstm_97 (LSTM)               (None, 30, 100)           80400     
_________________________________________________________________
lstm_98 (LSTM)               (None, 30, 100)           80400     
_________________________________________________________________
lstm_99 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 101       
Total params: 284,901
Trainable params: 284,901
Non-trainable params: 0
_________________________________________________________________
Train on 3800 samples, validate on 233 samples
Epoch 1/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 1550.9811 

3800/3800 [==============================] - 6s 1ms/step - loss: 1167.2634 - val_loss: 1162.9286
Epoch 66/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 1161.3102 - val_loss: 1156.9761
Epoch 67/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 1155.3583 - val_loss: 1151.0246
Epoch 68/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 1149.4074 - val_loss: 1145.0743
Epoch 69/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 1143.4575 - val_loss: 1139.1249
Epoch 70/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 1137.5084 - val_loss: 1133.1763
Epoch 71/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 1131.5601 - val_loss: 1127.2282
Epoch 72/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 1125.6123 - val_loss: 1121.2804
Epoch 73/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 1119.6651 - val_loss: 1115.3336
Epoch 74/1000
3

3800/3800 [==============================] - 6s 1ms/step - loss: 727.5241 - val_loss: 723.1949
Epoch 140/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 721.5815 - val_loss: 717.2524
Epoch 141/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 715.6390 - val_loss: 711.3098
Epoch 142/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 709.6965 - val_loss: 705.3672
Epoch 143/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 703.7540 - val_loss: 699.4248
Epoch 144/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 697.8116 - val_loss: 693.4824
Epoch 145/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 691.8691 - val_loss: 687.5399
Epoch 146/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 685.9266 - val_loss: 681.5975
Epoch 147/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 679.9890 - val_loss: 675.6616
Epoch 148/1000
3800/3800 

3800/3800 [==============================] - 6s 1ms/step - loss: 327.1433 - val_loss: 328.3437
Epoch 214/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 323.1182 - val_loss: 324.5309
Epoch 215/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 319.1459 - val_loss: 320.8397
Epoch 216/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 315.2054 - val_loss: 317.2091
Epoch 217/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 311.2997 - val_loss: 313.6197
Epoch 218/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 307.4293 - val_loss: 310.1096
Epoch 219/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 303.6221 - val_loss: 306.6269
Epoch 220/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 299.8787 - val_loss: 303.1278
Epoch 221/1000
3800/3800 [==============================] - 6s 1ms/step - loss: 296.2018 - val_loss: 299.6973
Epoch 222/1000
3800/3800 

3800/3800 [==============================] - 6s 2ms/step - loss: 196.9032 - val_loss: 211.5362
Epoch 288/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 196.7971 - val_loss: 211.4984
Epoch 289/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 196.7136 - val_loss: 211.4573
Epoch 290/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 196.6395 - val_loss: 211.4163
Epoch 291/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 196.5712 - val_loss: 211.3832
Epoch 292/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 196.5142 - val_loss: 211.3497
Epoch 293/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 196.4611 - val_loss: 211.3181
Epoch 294/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 196.4098 - val_loss: 211.2782
Epoch 295/1000
3800/3800 [==============================] - 6s 2ms/step - loss: 196.3607 - val_loss: 211.2532
Epoch 296/1000
3800/3800 

KeyboardInterrupt: 

In [339]:
X_trimmed_train = X_trimmed[:3800]
X_trimmed_test = X_trimmed[3800:]
Y_trimmed_train = Y_trimmed[:3800]
Y_trimmed_test = Y_trimmed[3800:]

model = Sequential()
model.add(LSTM(units=150, dropout = 0.5, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=150, dropout = 0.5, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=150, dropout = 0.5, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=150, dropout = 0.5, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=150, dropout = 0.5, return_sequences= True, input_shape=(30, 8)))
model.add(LSTM(units=100, dropout = 0.5))
model.add(Dense(units=1))
model.summary()
model.compile(optimizer='adam', loss='mean_absolute_error')

model.fit(X_trimmed_train, Y_trimmed_train, epochs=1000, batch_size=64, validation_data=(X_trimmed_test, Y_trimmed_test), shuffle=True)

Train on 3800 samples, validate on 233 samples
Epoch 1/1000
3800/3800 [==============================] - 13s 3ms/step - loss: 881.9229 - val_loss: 877.5643
Epoch 2/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 875.9233 - val_loss: 871.5650
Epoch 3/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 869.9237 - val_loss: 865.5653
Epoch 4/1000
3800/3800 [==============================] - 13s 3ms/step - loss: 863.9242 - val_loss: 859.5655
Epoch 5/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 857.9246 - val_loss: 853.5661
Epoch 6/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 851.9249 - val_loss: 847.5664
Epoch 7/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 845.9253 - val_loss: 841.5670
Epoch 8/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 839.9257 - val_loss: 835.5672
Epoch 9/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 83

3800/3800 [==============================] - 12s 3ms/step - loss: 446.5401 - val_loss: 443.9794
Epoch 76/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 441.1737 - val_loss: 438.7720
Epoch 77/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 435.8422 - val_loss: 433.6100
Epoch 78/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 430.5321 - val_loss: 428.4544
Epoch 79/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 425.2668 - val_loss: 423.3075
Epoch 80/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 420.0549 - val_loss: 418.2064
Epoch 81/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 414.8791 - val_loss: 413.1157
Epoch 82/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 409.7672 - val_loss: 408.1271
Epoch 83/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 404.7249 - val_loss: 403.1529
Epoch 84/1000
3800/3800 

3800/3800 [==============================] - 12s 3ms/step - loss: 207.1901 - val_loss: 219.3263
Epoch 150/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 206.2829 - val_loss: 218.6191
Epoch 151/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 205.4377 - val_loss: 218.0028
Epoch 152/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 204.6391 - val_loss: 217.4085
Epoch 153/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 203.8906 - val_loss: 216.8070
Epoch 154/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 203.1676 - val_loss: 216.2848
Epoch 155/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 202.5448 - val_loss: 215.8228
Epoch 156/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 202.0071 - val_loss: 215.4185
Epoch 157/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 201.5412 - val_loss: 215.0761
Epoch 158/1000
3

Epoch 223/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.3680 - val_loss: 211.1167
Epoch 224/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1613 - val_loss: 211.1047
Epoch 225/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.0816 - val_loss: 211.1012
Epoch 226/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1543 - val_loss: 211.0998
Epoch 227/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 197.6882 - val_loss: 211.1005
Epoch 228/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1495 - val_loss: 211.1060
Epoch 229/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1241 - val_loss: 211.1011
Epoch 230/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1853 - val_loss: 211.0967
Epoch 231/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1356 - val_loss: 211.0936
E

Epoch 297/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1345 - val_loss: 211.0976
Epoch 298/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.2024 - val_loss: 211.0980
Epoch 299/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1153 - val_loss: 212.4144
Epoch 300/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 197.6618 - val_loss: 212.0407
Epoch 301/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 197.2376 - val_loss: 211.6987
Epoch 302/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.9215 - val_loss: 211.5264
Epoch 303/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.7157 - val_loss: 211.4369
Epoch 304/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.5686 - val_loss: 211.3614
Epoch 305/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.4679 - val_loss: 211.3032
E

...

Epoch 667/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1281 - val_loss: 211.0942
Epoch 668/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1293 - val_loss: 211.0955
Epoch 669/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1292 - val_loss: 211.0944
Epoch 670/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1291 - val_loss: 211.0944
Epoch 671/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1319 - val_loss: 211.0960
Epoch 672/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1293 - val_loss: 211.0960
Epoch 673/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1318 - val_loss: 211.0962
Epoch 674/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1292 - val_loss: 211.0943
Epoch 675/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1341 - val_loss: 211.0954
E

Epoch 741/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1293 - val_loss: 211.0961
Epoch 742/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1321 - val_loss: 211.0962
Epoch 743/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1302 - val_loss: 211.0962
Epoch 744/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1291 - val_loss: 211.0955
Epoch 745/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1329 - val_loss: 211.0976
Epoch 746/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1307 - val_loss: 211.0982
Epoch 747/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1358 - val_loss: 211.0969
Epoch 748/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1340 - val_loss: 211.0944
Epoch 749/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1338 - val_loss: 211.0976
E

Epoch 815/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1299 - val_loss: 211.0958
Epoch 816/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1306 - val_loss: 211.0952
Epoch 817/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1295 - val_loss: 211.0961
Epoch 818/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1297 - val_loss: 211.0944
Epoch 819/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1285 - val_loss: 211.0962
Epoch 820/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1310 - val_loss: 211.0948
Epoch 821/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1291 - val_loss: 211.0958
Epoch 822/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1299 - val_loss: 211.0966
Epoch 823/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1320 - val_loss: 211.0961
E

Epoch 889/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1044 - val_loss: 211.0931
Epoch 890/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1114 - val_loss: 211.0932
Epoch 891/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1146 - val_loss: 211.0947
Epoch 892/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1378 - val_loss: 211.0937
Epoch 893/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1160 - val_loss: 211.0937
Epoch 894/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1152 - val_loss: 211.0929
Epoch 895/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1334 - val_loss: 211.0940
Epoch 896/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1273 - val_loss: 211.0941
Epoch 897/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1288 - val_loss: 211.0959
E

Epoch 963/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1134 - val_loss: 211.0985
Epoch 964/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1404 - val_loss: 211.0977
Epoch 965/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1427 - val_loss: 211.0979
Epoch 966/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1257 - val_loss: 211.0967
Epoch 967/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1255 - val_loss: 211.0972
Epoch 968/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1146 - val_loss: 211.0955
Epoch 969/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.0970 - val_loss: 211.0964
Epoch 970/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1438 - val_loss: 211.0966
Epoch 971/1000
3800/3800 [==============================] - 12s 3ms/step - loss: 196.1304 - val_loss: 211.0967
E